In [1]:
import os
import pandas as pd

In [2]:
df1=pd.read_csv("C:/downloads/application-failure-prediction-part1/train_data.csv")
print(df1.head())
print(df1.shape)

         job_id  memory_GB  network_log10_MBps  local_IO_log10_MBps  \
0  jobID1634295    44.3904             -1.0262               0.8033   
1  jobID2033452    31.5839             -1.4608              -0.6080   
2  jobID2068800   154.4610             -0.5508              -0.3637   
3    jobID78826    71.8570              3.5701              -0.1281   
4  jobID2935014    30.0370              3.3094              -0.9463   

   NFS_IO_log10_MBps  failed  
0            -3.0000       0  
1            -2.9967       0  
2            -3.0000       0  
3             2.3040       0  
4            -3.0000       0  
(20000, 6)


In [3]:
# !pip install scikit-learn
# !pip install matplotlib
# !pip install cvxopt

In [4]:
# Utility
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.preprocessing import StandardScaler

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
import pickle 

#seed
seed = 12345

train = df1.iloc[:, 1:-1]
target = df1.iloc[:, -1]
scaled_train = StandardScaler().fit_transform(train)

# Splitting the data into training and validation
X_train, X_test, y_train, y_test = train_test_split(
    train, target, test_size=0.20, random_state=seed
)

# building the all models
model_1 = RandomForestClassifier(random_state=seed)
model_2 = LogisticRegression(random_state=seed, max_iter=1000)
model_3 = SGDClassifier(random_state=seed)
model_4 = KNeighborsClassifier()
model_5 = AdaBoostClassifier(random_state=seed)
#model_6 = svm.SVC(kernel='linear')  #too slow



# training
model_1.fit(X_train, y_train)
model_2.fit(X_train, y_train)
model_3.fit(X_train, y_train)
model_4.fit(X_train, y_train)
model_5.fit(X_train, y_train)
#model_6.fit(X_train, y_train)

estimators=[("rf", model_1), ("lr", model_2), ("sgd", model_3), ("knn", model_4),("ada", model_5)] #, ("svm", model_6)]

final_model = VotingClassifier(
    estimators=estimators,
    voting="hard",
)

# training
final_model.fit(X_train, y_train)

pickle.dump(final_model, open("../models/voting", 'wb')) 

# prediction
prediction = final_model.predict(X_test)

# evaluation
print("*****Voting****\n")
accuracy = round(accuracy_score(y_test, prediction) * 100, 3)
auc = round(roc_auc_score(y_test, prediction), 3)


print(f" Accuracy: {accuracy}%")
print(f" AUC score: {auc}")

# building the model
final_estimator = GradientBoostingClassifier(random_state=seed)

# Stacking Classifier
clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)

# training
clf.fit(X_train, y_train)

pickle.dump(final_model, open("../models/stacking", 'wb')) 

# prediction
prediction = clf.predict(X_test)

# evaluation
print("*****stacking****\n")
accuracy = round(accuracy_score(y_test, prediction) * 100, 3)
auc = round(roc_auc_score(y_test, prediction), 3)

print(f" Accuracy: {accuracy}%")
print(f" AUC score: {auc}")



*****Voting****

 Accuracy: 93.325%
 AUC score: 0.565
*****stacking****

 Accuracy: 94.65%
 AUC score: 0.689


# unsupervised approach

In [5]:
# from OCC_class import OCC, OkMeans, OSVM, OSVM2, OkNN, dbscan
# # building the all models
# model_1 = OkMeans(train, step = 0.05, eps = 0.005, labels=None, y=target)
# model_2 = OSVM(train, step = 0.05, eps = 0.005, labels=None, y=target)
# model_3 = OkNN(train, step = 0.05, eps = 0.005, labels=None, y=target)
# # model_ 4= OSVM2(train, step = 0.05, eps = 0.005, labels=None, y=target)
# # model_5 = dbscan(train, step = 0.05, eps = 0.005, labels=None, y=target)

# model_1.fit(thresh = 0.9, k = 1, sample_weight=1)
# nu = 0.001
# sigma2 = 20
# weights=1


# model_2.fit(nu, sigma2, weights)
# model_3.fit(thresh = 0.9, k = 1)

# # nu = 0.001
# # sigma2 = 20
# # model_4.fit(nu, sigma2)

# n_estimators=[("kmeans", model_1), ("osvm", model_2), ("knn", model_3)] #, ("osvm2", model_4),("dbscan", model_5)] ]

# final_model = VotingClassifier(
#     estimators=n_estimators,
#     voting="hard",
# )

# # training
# final_model.fit(X_train, y_train)

# # prediction
# prediction = final_model.predict(X_test)

# # evaluation
# print("*****Voting****\n")
# accuracy = round(accuracy_score(y_test, prediction) * 100, 3)
# auc = round(roc_auc_score(y_test, prediction), 3)



# print(f" Accuracy: {accuracy}%")
# print(f" AUC score: {auc}")

